In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Nueva sección

In [ ]:
import torch

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-5-27 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
def calculateIOU(box_f1, box_f2):
  # Calculate area of each box
  area_box1 = (box_f1[2] - box_f1[0]) * (box_f1[3] - box_f1[1])
  area_box2 = (box_f2[2] - box_f2[0]) * (box_f2[3] - box_f2[1])

  # Area of intersection:
  x_inter1 = max(box_f1[0], box_f2[0])
  y_inter1 = max(box_f1[1], box_f2[1])
  x_inter2 = min(box_f1[2], box_f2[2])
  y_inter2 = min(box_f1[3], box_f2[3])
  area_inter = (x_inter2 - x_inter1) * (y_inter2 - y_inter1)
        
  # IOU Calculation:
  area_union = area_box1 + area_box2 - area_inter
  iou = area_inter / area_union
  #print("(IOU): ", iou)
  return iou


def detectMovements(cameraResults):
  movementDetected = []
  iou_list = []
  for i in range(len(cameraResults[0])):
    iou = calculateIOU(cameraResults[0][i], cameraResults[1][i])
    iou_list.append(iou)
    if (iou < 0.75):
      #print("Person ", i, " has moved!")
      movementDetected.append(True)
    else:
      movementDetected.append(False)
  
  return movementDetected, iou_list

In [ ]:
def detectImageMovement(path1,path2):
  # Images
  imgs = [path1,path2] 

  # Inference
  results = model(imgs)

  res_list, iou_list = detectMovements(results.xyxy[0].detach().cpu().numpy())
  
  print(res_list)

In [ ]:
p1 = '

1persona01.jpg'
p2 = '../content/drive/MyDrive/Colab Notebooks/Yolo code/1persona02.jpg'
detectImageMovement(p1,p2)

(IOU):  tensor(0.51120, device='cuda:0')
Person  0  has moved!
[True]


In [ ]:
p1 = '../content/drive/MyDrive/Colab Notebooks/Yolo code/2persones01.jpg'
p2 = '../content/drive/MyDrive/Colab Notebooks/Yolo code/2persones02.jpg'
detectImageMovement(p1,p2)

(IOU):  tensor(0.83709, device='cuda:0')
(IOU):  tensor(0.54260, device='cuda:0')
Person  1  has moved!
[False, True]


In [ ]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


def drawBox(img, pts1, pts2):
  # im is a PIL Image object
  im_arr = np.asarray(img)

  # convert rgb array to opencv's bgr format
  im_arr_bgr = cv2.cvtColor(im_arr, cv2.COLOR_RGB2BGR)

  # pts1 and pts2 are the upper left and bottom right coordinates of the rectangle
  rand_r = random.randint(0,255)
  rand_g = random.randint(0,255)
  rand_b = random.randint(0,255)
  cv2.rectangle(im_arr_bgr, pts1, pts2, color=(rand_r, rand_g, rand_b), thickness=2)
  im_arr = cv2.cvtColor(im_arr_bgr, cv2.COLOR_BGR2RGB)

  return im_arr

def drawBoxes(imgs):
  img_list = []
  results = model(imgs)
  res = results.xyxy[0].detach().cpu().numpy()
  img = cv2.imread(imgs)
  for result in res:
    point1 = (result[0], result[1])
    point2 = (result[2], result[3])
    img = drawBox(img, point1, point2)
  return img

In [ ]:
p2 = '../content/drive/MyDrive/Colab Notebooks/Yolo code/2persones01.jpg'
img_res = drawBoxes(p2)
cv2.imwrite('2persones01_res.jpg', img_res)

True

In [ ]:
p2 = '../content/drive/MyDrive/Colab Notebooks/Yolo code/1persona01.jpg'
img_res = drawBoxes(p2)
cv2.imwrite('1persona01_res.jpg', img_res)

True

In [ ]:

def avrgColor(img, p1, p2):
  aux1 = img[p1[1]:p2[1],p1[0]:p2[0]]
  sizeBox = aux1.shape
  aux2 = aux1[int(0.3*sizeBox[0]):int(0.7*sizeBox[0]), int(0.3*sizeBox[1]):int(0.7*sizeBox[1])]
  avrg = np.average(aux2, axis=0)
  avrg = np.average(avrg, axis=0)
  return avrg

def getAverages(results, path):
  avrg_list = []
  res = results.xyxy[0].detach().cpu().numpy()
  img = cv2.imread(path)

  for result in res:
    p1 = (int(result[0]), int(result[1]))
    p2 = (int(result[2]), int(result[3]))
    avrg_list.append(avrgColor(img, p1, p2))

  return avrg_list

def closestAverage(avg_list, new_avg_list, results):
  shape_average = np.array(avg_list).shape
  shape_results = np.array(results).shape
  ret = np.zeros(shape_average)
  aux = np.zeros(shape_results)
  print(ret.shape)
  print(aux.shape)
  print(len(new_avg_list))
  print(len(avg_list))
  for i in range(len(new_avg_list)):
    score = [0 for i in range(len(new_avg_list))] 
    #print(" For: ", i)
    for j in range(len(avg_list)):
      score[j] += abs(avg_list[j][0]-new_avg_list[i][0])
      score[j] += abs(avg_list[j][0]-new_avg_list[i][0])
      score[j] += abs(avg_list[j][0]-new_avg_list[i][0])
      #print("   Score: ", score[j])
    #print(" Moves to: ", np.argmin(score))
    ret[np.argmin(score)] = new_avg_list[i]
    aux[np.argmin(score)] = results[i]
  return ret, aux


In [ ]:
playerAverageList = []
playerNameList = []

def addPlayer(path, name):
  playerAverageList.append(getAverages(model(path), path)[0])
  playerNameList.append(name)

def selectDeletedPlayers(movement):
  ret = []
  for i in range(len(movement)):
    if movement[i]:
      ret.append(i)
  return ret

def deletePlayer(index):
  a = playerAverageList
  b = playerNameList
  for i in reversed(range(len(index))):
    a = np.delete(a, index[i])
    b = np.delete(b, index[i])
  return a, b

In [ ]:
playerAverageList = []
playerNameList = []

#===========================================================================================================#
#                                         #Inicialitzar jugadors                                            #
#===========================================================================================================#


player1path = '../content/drive/MyDrive/Colab Notebooks/Yolo code/1persona01.jpg'
player1name = 'Player0'
addPlayer(player1path, player1name)

#player2path = '../content/drive/MyDrive/Colab Notebooks/Yolo code/1personatest.jpg'
#player2name = 'Player1'
#addPlayer(player2path, player2name)

print(playerNameList)

#===========================================================================================================#
#                                       #Primer frame i posicions:                                          #
#===========================================================================================================#


path = '../content/drive/MyDrive/Colab Notebooks/Yolo code/1persona01.jpg' 
results = model(path)
avg_list = getAverages(results, path)

playerAverageList, bBoxes1  = closestAverage(playerAverageList, avg_list, results.xyxy[0].detach().cpu().numpy())

#print("Preparing frames data:")
#print(new_avg_list1)
#print(new_avg_list2)
#print(playerAverageList)
#print(bBoxes1)
#print(bBoxes2)

#===========================================================================================================#
#                                              MainLooop                                                    #
#===========================================================================================================#

new_path = '../content/drive/MyDrive/Colab Notebooks/Yolo code/1persona02.jpg' 
new_results = model(new_path)
new_avg_list = getAverages(new_results, new_path)

playerAverageList, bBoxes2 = closestAverage(playerAverageList, new_avg_list, new_results.xyxy[0].detach().cpu().numpy())

movement_list, iou_list = detectMovements((bBoxes1, bBoxes2))
index = selectDeletedPlayers(movement_list)
playerAverageList, playerNameList = deletePlayer(index)

bBoxes1 = bBoxes2

#print(movement_list)
#print(index)
print(playerNameList)

['Player0']
(1, 3)
(1, 6)
1
1
(1, 3)
(1, 6)
1
1
[]
